In [219]:
import requests
import json
import sqlite3

## Project: looking at my top 100 songs
I am going to use Spotify's API to access 4 playlists: my top 100 songs from 2022, 2023, 2024, and 2025 (migrated from Apple Music). Then, I am going to create different SQL tables with to explore relationships between all of my favorite songs from the past 4 years.

In [ ]:
# Request a bearer acess token given my Spotify client id / client secret
url = "https://accounts.spotify.com/api/token"
data = {
    "grant_type": "client_credentials",
    "client_id": "5a129cbf9b274bffb917704d3c4600b5",
    "client_secret": "d98c365ee0cd43bf8257cbfa27857245",
}

response = requests.post(url, data=data)
authorization = f"Bearer {response.json()['access_token']}"


# Using that authorization, access each playlist
# Turn it into text, then turn it into a dictorionary using json.loads

# For 2022
playlist_2022 = "https://api.spotify.com/v1/playlists/71zUjPqRk9OxTqpXYEmzlA/tracks"
playlist_2022_info = requests.get(url=playlist_2022, headers={"Authorization": authorization})
res_2022 = json.loads(playlist_2022_info.text)

# For 2023
playlist_2023 = "https://api.spotify.com/v1/playlists/28b3xd3RpAvarZB5PKr6Nx/tracks"
playlist_2023_info = requests.get(
    url=playlist_2023, headers={"Authorization": authorization}
)
res_2023 = json.loads(playlist_2023_info.text)

# For 2024
playlist_2024 = "https://api.spotify.com/v1/playlists/5P6ptyrOrEwoxn7A3qhC7F/tracks"
playlist_2024_info = requests.get(
    url=playlist_2024, headers={"Authorization": authorization}
)
res_2024 = json.loads(playlist_2024_info.text)

# For 2025 (so far)
playlist_2025 = "https://api.spotify.com/v1/playlists/0VLFZelqY8r0g544QAy8zv/tracks"
playlist_2025_info = requests.get(
    url=playlist_2025, headers={"Authorization": authorization}
)
res_2025 = json.loads(playlist_2025_info.text)

### My SQL Plan:
- Create table PLAYLISTS to hold information about my four playlists (top 100 songs for 2022, 2023, 2024, 2025)
- Create table SONGS to hold information about the unique songs in all of these playlists
- Create table SONG ON PLAYLIST to relate the two tables above (song ID and playlist ID will relate this)

In [ ]:
conn = sqlite3.connect('favorite_songs.db')
cursor = conn.cursor()

# cursor.execute("""CREATE TABLE playlists (
#                id text,
#                name text
#                )""")

# cursor.execute("INSERT INTO playlists VALUES ('71zUjPqRk9OxTqpXYEmzlA', 'Replay 2022')")
# cursor.execute("INSERT INTO playlists VALUES ('28b3xd3RpAvarZB5PKr6Nx', 'Replay 2023')")
# cursor.execute("INSERT INTO playlists VALUES ('5P6ptyrOrEwoxn7A3qhC7F', 'Replay 2024')")
# cursor.execute("INSERT INTO playlists VALUES ('0VLFZelqY8r0g544QAy8zv', 'Replay 2025')")

# conn.commit()

In [ ]:
def get_song_info(playlist_res):
    """Given the dictionary created above, access all of this information
    about each song (should be 100 songs) on the playlist.
    Return the result as a list of tuples for each song"""

    # Holds all of the tuples, one tuple for each song
    song_info = []

    # Iterate through each song
    for i in range(0, len(playlist_res["items"])):
        song_name = playlist_res["items"][i]["track"]["name"]
        song_id = playlist_res["items"][i]["track"]["id"]
        song_artist = playlist_res["items"][i]["track"]["artists"][0]["name"]
        song_runtime = playlist_res["items"][i]["track"]["duration_ms"]
        song_popularity = playlist_res["items"][i]["track"]["popularity"]
        song_album = playlist_res["items"][i]["track"]["album"]["name"]

        # Tuple holding song's info gets appended
        full_info = (song_id, song_name, song_artist, song_runtime, song_popularity, song_album)
        song_info.append(full_info)

    return song_info

In [ ]:
# cursor.execute(
#   """DROP TABLE Songs"""
# )

In [ ]:
# cursor.execute(
#     """CREATE TABLE Songs (
#     id text,
#     name text,
#     artist text,
#     runtime real,
#     popularity integer,
#     album text
#     )"""
# )
# conn.commit()


def insert_song_info(playlist_res):
    for i in range(len(playlist_res["items"])):
        id, name, artist, runtime, popularity, album = get_song_info(playlist_res)[i]

        # Insert only if the song does not exist
        cursor.execute(
            """
            INSERT INTO Songs (id, name, artist, runtime, popularity, album)
            SELECT ?, ?, ?, ?, ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM Songs WHERE id = ?
            )
            """,
            (id, name, artist, runtime, popularity, album, id),
        )

        conn.commit()
        print(f"Inserted song {name}")


# Insert songs for multiple years
insert_song_info(res_2022)
insert_song_info(res_2023)
insert_song_info(res_2024)
insert_song_info(res_2025)

Inserted song Afterglow
Inserted song august
Inserted song Out Of The Woods
Inserted song Cornelia Street
Inserted song peace
Inserted song Delicate
Inserted song End Game
Inserted song Lover
Inserted song Don't Know Why
Inserted song Everything Has Changed (feat. Ed Sheeran) (Taylor’s Version)
Inserted song Come Away With Me
Inserted song Cruel Summer
Inserted song Higher
Inserted song Turn Me On
Inserted song Sunrise
Inserted song Sweet Nothing
Inserted song Pipe Down
Inserted song Yebba’s Heartbreak
Inserted song The Archer
Inserted song BRIGHTSIDE
Inserted song Feathered Indians
Inserted song Paper Rings
Inserted song Maroon
Inserted song cowboy like me
Inserted song New Year’s Day
Inserted song Dancing With Our Hands Tied
Inserted song All Falls Down
Inserted song Daylight
Inserted song Get It Together
Inserted song Race My Mind
Inserted song Grapejuice
Inserted song Cornelia Street - Live From Paris
Inserted song Fireworks
Inserted song ivy
Inserted song the last great american d

In [ ]:
# cursor.execute(
#   """CREATE TABLE Songs_On_Playlist (
#   Song_ID text,
#   Playlist_ID text)"""
# )

def insert_id_info(playlist_res, playlist_id):
    """Insert song - playlist ids to establish relationship"""
    
    for i in range(len(playlist_res["items"])):
        song_id = get_song_info(playlist_res)[i][0]

        # Insert only if the song-playlist pair does not already exist
        cursor.execute(
            """
            INSERT INTO Songs_On_Playlist (song_id, playlist_id)
            SELECT ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM Songs_On_Playlist WHERE song_id = ? AND playlist_id = ?
            )
            """,
            (song_id, playlist_id, song_id, playlist_id),
        )

        conn.commit()


# Insert song-playlist relationships
insert_id_info(res_2022, "71zUjPqRk9OxTqpXYEmzlA")
insert_id_info(res_2023, "28b3xd3RpAvarZB5PKr6Nx")
insert_id_info(res_2024, "5P6ptyrOrEwoxn7A3qhC7F")
insert_id_info(res_2025, "0VLFZelqY8r0g544QAy8zv")

## Exploration
Now that I have these three tables, I would like to do some exploration.

In [252]:
# Get the average popularity of
popularity = cursor.execute(
  """SELECT Playlists.name AS playlist_name, AVG(Songs.popularity) AS avg_popularity
  FROM Songs
  JOIN Songs_On_Playlist ON Songs.id = Songs_On_Playlist.Song_ID
  JOIN Playlists ON Songs_On_Playlist.Playlist_ID = Playlists.id
  GROUP BY Playlists.id
  ORDER BY avg_popularity DESC"""
)
popularity.fetchall()

[('Replay 2022', 60.11),
 ('Replay 2023', 54.16),
 ('Replay 2024', 49.505050505050505),
 ('Replay 2025', 45.38)]

This result suggests that when I was first getting into music in 2022, I listened to more mainstream songs that are still popular today. Then as the years went on, I started refining my music taste.

In [256]:
# What songs have been on my replay since 2022?
# Basically, I want the songs which have a repeat entry in Songs_On_Playlist (same SongID, different playlistID)
consistent_songs = cursor.execute(
    """SELECT Songs.name AS song_name, COUNT(*) as Freq
  FROM Songs
  JOIN Songs_On_Playlist ON Songs.id = Songs_On_Playlist.Song_ID
  GROUP BY Songs_On_Playlist.Song_ID
  ORDER BY Freq DESC"""
)
consistent_songs.fetchall()

[('First Time', 3),
 ('Autumn Leaves - Album Version - (Take 1)', 3),
 ('Jackie And Wilson', 3),
 ('august', 3),
 ('cowboy like me', 3),
 ('Part Of The Band', 3),
 ('Wintering', 3),
 ('Happiness', 3),
 ('Drops of Jupiter (Tell Me)', 3),
 ("September In The Rain - Live At Mister Kelly's, Chicago / 1957", 3),
 ('Daylight', 3),
 ('About You', 3),
 ('Lover', 3),
 ('All I Need To Hear', 3),
 ('Cruel Summer', 3),
 ('Oh Caroline', 3),
 ("I'm In Love With You", 3),
 ('Sunrise', 3),
 ('Looking For Somebody (To Love)', 3),
 ('De Selby (Part 2)', 3),
 ('Unknown / Nth', 3),
 ('Harvest Moon', 2),
 ('The 1975', 2),
 ('Autumn In Rome', 2),
 ('Robbers', 2),
 ('The Sound', 2),
 ('So High School', 2),
 ("'Tis Autumn", 2),
 ('Valerie (feat. Amy Winehouse) - Version Revisited', 2),
 ('Tears Dry On Their Own', 2),
 ('Misty', 2),
 ('Autumn In New York', 2),
 ('All Things End', 2),
 ('Come Away With Me', 2),
 ('Francesca', 2),
 ("You Don't Know My Name", 2),
 ('Everything Has Changed (feat. Ed Sheeran) (Tayl